In [1]:
# Importing necessary libraries
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from scipy.spatial import distance_matrix
import pulp

# Configure osmnx to use cache
ox.config(use_cache=True)


/var/folders/17/xvl07b2n1kq7qscwxp63zc680000gn/T/ipykernel_71370/4233648237.py:10: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(use_cache=True)


In [10]:
# Define the city or area
place_name = "Manhattan, New York City, New York, USA"

# Fetch POIs
tags = {'amenity': 'recycling'}  # Change this tag based on the facilities you're interested in
gdf = ox.geometries_from_place(place_name, tags)




/var/folders/17/xvl07b2n1kq7qscwxp63zc680000gn/T/ipykernel_71370/3610195962.py:6: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(place_name, tags)


In [13]:
# Ensure all geometries are points (convert polygons to centroids if necessary)
gdf['geometry'] = gdf['geometry'].apply(lambda x: x.centroid if x.geom_type != 'Point' else x)


In [14]:
gdf['x'] = gdf.geometry.x

In [15]:
# Calculate coordinates

coords = [(point.x, point.y) for point in gdf.geometry]

# Compute the Euclidean distance matrix
dist_matrix = distance_matrix(coords, coords)

# Create a distance dictionary suitable for the optimization model
distances = {(i, j): dist_matrix[i][j] for i in range(len(coords)) for j in range(len(coords)) if i != j}

In [16]:
# Setting up the linear programming model
prob = pulp.LpProblem("CityWasteCollection", pulp.LpMinimize)

# Create variables for each route:
routes = pulp.LpVariable.dicts("Route", distances, 0, 1, pulp.LpBinary)

# Objective function: Minimize the total distances traveled
prob += pulp.lpSum([distances[(i, j)] * routes[(i, j)] for (i, j) in distances]), "Total_Distance"

# Constraints:
# Ensure that each location is visited exactly once coming and going
for location in range(len(coords)):
    prob += pulp.lpSum([routes[(i, j)] for i, j in routes if j == location]) == 1, f"Enter_{location}"
    prob += pulp.lpSum([routes[(i, j)] for i, j in routes if i == location]) == 1, f"Exit_{location}"


In [17]:
# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/17/xvl07b2n1kq7qscwxp63zc680000gn/T/29a7e9213ac243e098dc2674e775a1af-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/17/xvl07b2n1kq7qscwxp63zc680000gn/T/29a7e9213ac243e098dc2674e775a1af-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 453 COLUMNS
At line 250214 RHS
At line 250663 BOUNDS
At line 300616 ENDATA
Problem MODEL has 448 rows, 49952 columns and 99904 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 48201.5 - 0.13 seconds
Cgl0004I processed model has 448 rows, 49952 columns (49952 integer (49952 of which binary)) and 99904 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 48201.5
Cbc0038I Bef

1

In [18]:
# Output the results:
print("Status:", pulp.LpStatus[prob.status])
routes_used = [(i, j) for i, j in routes if pulp.value(routes[(i, j)]) == 1]
print("Routes used:", routes_used)


Status: Optimal
Routes used: [(0, 8), (1, 11), (2, 162), (3, 12), (4, 129), (5, 67), (6, 50), (7, 49), (8, 0), (9, 219), (10, 54), (11, 1), (12, 20), (13, 14), (14, 15), (15, 13), (16, 33), (17, 22), (18, 214), (19, 30), (20, 3), (21, 36), (22, 17), (23, 24), (24, 23), (25, 26), (26, 25), (27, 37), (28, 59), (29, 31), (30, 19), (31, 29), (32, 28), (33, 16), (34, 35), (35, 34), (36, 40), (37, 27), (38, 39), (39, 38), (40, 21), (41, 200), (42, 43), (43, 42), (44, 176), (45, 63), (46, 47), (47, 46), (48, 177), (49, 7), (50, 58), (51, 52), (52, 51), (53, 57), (54, 10), (55, 56), (56, 122), (57, 53), (58, 6), (59, 32), (60, 61), (61, 62), (62, 60), (63, 45), (64, 65), (65, 66), (66, 64), (67, 5), (68, 80), (69, 70), (70, 69), (71, 72), (72, 71), (73, 86), (74, 81), (75, 78), (76, 77), (77, 76), (78, 75), (79, 102), (80, 68), (81, 74), (82, 83), (83, 82), (84, 85), (85, 84), (86, 73), (87, 88), (88, 87), (89, 90), (90, 89), (91, 100), (92, 98), (93, 95), (94, 99), (95, 93), (96, 97), (97, 96

In [22]:
import folium

# Creating a map using folium
map = folium.Map(location=[np.mean([y for _, y in coords]), np.mean([x for x, _ in coords])], zoom_start=13, tiles='cartodbpositron')

# Add points to the map
for point in coords:
    folium.CircleMarker(location=[point[1], point[0]], radius=5, color='blue', fill=True).add_to(map)


In [23]:
# Draw routes on the map
for i, j in routes_used:
    start, end = coords[i], coords[j]
    line = folium.PolyLine(locations=[[start[1], start[0]], [end[1], end[0]]], color='red', weight=5)
    map.add_child(line)

# Display the map
map